# Data Processing

***

**Owner:** Ling-Yun, Huang

**Project Title:** Enhance Road Safety, Focusing on Pedestrian: Using Taipei Traffic Accident Data in 2020-2022.


***

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import 2020-2022 datasets
df2020 = pd.read_csv("Taipei_accident_2020.csv")
df2021 = pd.read_csv("Taipei_accident_2021.csv")
df2022 = pd.read_csv("Taipei_accident_2022.csv", low_memory=False)
raw_data = pd.concat([df2020,df2021,df2022], ignore_index=True)
raw_data.shape

(172446, 50)

In [3]:
# delete the accident that not in A1 or A2 (no death or injury)
drop_rows = ((raw_data['Death_24h']==0) & (raw_data['Death_30d']==0) & (raw_data['Injure']==0))
df_pre = raw_data[~drop_rows]
len(df_pre)

172444

In [4]:
# display sex variable
sex_counts = df_pre.Sex.value_counts()
sex_percentages = df_pre.Sex.value_counts(normalize=True)*100
sex_summary = pd.concat([sex_counts,sex_percentages], axis=1)
sex_summary.columns = ['Count', 'Percentage']
codebook = {1:'Male', 2:'Female', 3:'Property', 4:'Unknown'}
sex_summary['Codebook'] = sex_summary.index.map(codebook.get)

print('Missing value in Sex is:', df_pre.Sex.isna().sum())
sex_summary

Missing value in Sex is: 0


,Count,Percentage,Codebook
Sex,,,
1,114679,66.502169,Male
2,52139,30.235323,Female
3,3895,2.258704,Property
4,1731,1.003804,Unknown


Since this research only focusing on human, sex = 3,4 would be drop in later analysis.

In [5]:
df_individual = df_pre.drop(df_pre[df_pre['Sex'].isin([3,4])].index)
len(df_individual)

166818

In [6]:
# display individual's vehicle use category
Parties_category_counts = df_individual.Parties_category.value_counts()
Parties_category_percentages = df_individual.Parties_category.value_counts(normalize=True)*100
Parties_category_summary = pd.concat([Parties_category_counts,Parties_category_percentages], axis=1)
Parties_category_summary.columns = ['Count', 'Percentage']
codebook = {
    'A01':'Public City Bus','A02':'Private City Bus','A03':'Public Intercity Bus',
    'A04':'Private Intercity Bus','A05':'Tourist Bus','A06':'Private bus',
    'A11':'Heavy trucks for business use','A12':'Heavy trucks for own use',
    'A21':'Tractor-full Trailer for business use','A22':'Tractor-full Trailer for own use',
    'A31':'Tractor–semi Trailer for business use','A32':'Tractor–semi Trailer for own use',
    'A41':'Tractor for business use','A42':'Tractor for own use',
    'B01':'Taxi','B02':'Sedan - Rental Car','B03':'Sedan - own use',
    'B11':'Small Truck for business use','B12':'Small Truck for own use',
    'C01':'Motorcycle (>550C.C.)','C02':'Motorcycle (250-550C.C.)','C03':'Ordinary Heavy Scooter(50-250C.C.)',
    'C04':'Ordinary Light Scooter(<50C.C.)','C05':'Small Light Scooter',
    'D01':'Military vehicles - Bus','D03':'Military vehicles - Sedan',
    'E01':'Ambulance','E02':'Fire Truck','E03':'Police Car',
    'E04':'Engineering vehicle','E05':'Other special vehicles',
    'F01':'Bicycle','F02':'Electric assisted bicycle','F03':'Electric bicycle',
    'F04':'Rickshaw','F06':'Other slow vehicle',
    'G01':'Assembled car','G02':'Farming vehicles','G03':'Power mechanical vehicle',
    'G04':'Trailer','G06':'Other vehicle',
    'H01':'Pedestrian','H02':'Passenger','H03':'Other people'
           }
Parties_category_summary['Codebook'] = Parties_category_summary.index.map(codebook.get)
Parties_category_summary.sort_index()

,Count,Percentage,Codebook
Parties_category,,,
A01,41,0.024578,Public City Bus
A02,1326,0.794878,Private City Bus
A03,21,0.012589,Public Intercity Bus
A04,387,0.231989,Private Intercity Bus
A05,126,0.075531,Tourist Bus
A06,19,0.011390,Private bus
A11,165,0.098910,Heavy trucks for business use
A12,349,0.209210,Heavy trucks for own use
A21,7,0.004196,Tractor-full Trailer for business use


In [7]:
## Create a new column to transform the individual vehicle use categories
grouping = [(df_individual.Parties_category.isin(['A01','A02','A03','A04','A05','A06','A11',
                                                  'A12','A21','A22','A31','A32','A41','A42'])),
            (df_individual.Parties_category.isin(['B01','B02','B03','B11','B12'])),
            (df_individual.Parties_category.isin(['C01','C02','C03','C04','C05'])),
            (df_individual.Parties_category.isin(['F01','F02','F03'])),
            (df_individual.Parties_category == 'H01'),
            (df_individual.Parties_category.isin(['H02','H03']))]

# 1:'Bus/Truck',2:'Sedan/Small Truck',3:'Motorcycle/Scooter',4:'Bicycle',5:'Pedestrian',
# 6:'Other vehicle',7:'Passenger/other people'
new_group = [1,2,3,4,5,7]

df_individual['Category_grouping'] = np.select(grouping, new_group, default=6)

# show count table in vehicle groups
new_Parties_category_counts = df_individual['Category_grouping'].value_counts()
new_Parties_category_percentages = df_individual['Category_grouping'].value_counts(normalize=True)*100
new_Parties_category_summary = pd.concat([new_Parties_category_counts,new_Parties_category_percentages], axis=1)
new_Parties_category_summary.columns = ['Count', 'Percentage']
codebook = {1:'Bus/Truck',2:'Sedan/Small Truck',3:'Motorcycle/Scooter',4:'Bicycle',5:'Pedestrian',
            6:'Other vehicle',7:'Passenger/other people'}
new_Parties_category_summary['Name'] = new_Parties_category_summary.index.map(codebook.get)
new_Parties_category_summary.sort_index()

,Count,Percentage,Name
Category_grouping,,,
1,2580,1.546596,Bus/Truck
2,49959,29.948207,Sedan/Small Truck
3,91890,55.083984,Motorcycle/Scooter
4,4450,2.667578,Bicycle
5,6495,3.893465,Pedestrian
6,330,0.197820,Other vehicle
7,11114,6.662351,Passenger/other people


For overall data, we will create both individual-based and accident-based datasets.

In [8]:
# Add series number for each accident
df_individual['Accident_no'] = df_individual.groupby(['Year','Month','Day','Hours','Minutes','Accident_type',
                                                      'Longitude_x','Latitude_y']).ngroup() + 1

# Create different columns that show accident's involve scooter, bicycle, or pedestrian
df_accident_pre = df_individual.copy()
df_accident_pre['involve_bus'] = (df_accident_pre.Category_grouping == 1).astype(int)
df_accident_pre['involve_car'] = (df_accident_pre.Category_grouping == 2).astype(int)
df_accident_pre['involve_scooter'] = (df_accident_pre.Category_grouping == 3).astype(int)
df_accident_pre['involve_bicycle'] = (df_accident_pre.Category_grouping == 4).astype(int)
df_accident_pre['involve_pedestrian'] = (df_accident_pre.Category_grouping == 5).astype(int)
df_accident_pre['involve_other'] = (df_accident_pre.Category_grouping == 6).astype(int)
df_accident_pre['involve_people'] = (df_accident_pre.Category_grouping == 7).astype(int)

df_involve_a = df_accident_pre.groupby('Accident_no',as_index=False).sum()[['Accident_no','involve_bus',
                                                        'involve_car','involve_scooter','involve_bicycle',
                                                        'involve_pedestrian','involve_other','involve_people']]

# All accidents columns
accident_col = ['Year','Month','Day','Hours','Minutes','Category','District', 
                'Death_24h','Death_30d','Injure','Weather','Light','Road_category','Speed_limit',
                'Road_type','Accident_site','Road_condition1','Road_condition2','Road_condition3',
                'Road_obstacle1','Road_obstacle2','Traffic_sign1','Traffic_sign2',
                'Lane_division','Lane_division_1','Lane_division_2','Lane_division_3', 
                'Accident_type','Cause_code_major','Longitude_x','Latitude_y','Accident_no']

# Create a accident-based data
df_accident = df_accident_pre.drop_duplicates(subset='Accident_no')[accident_col]

# Create a column in Fatal or not by accident
conditions = (df_accident.Death_24h > 0) | (df_accident.Death_30d > 0)
df_accident['Fatal'] = conditions.astype(int)

# Merge involve scooter, bicycle, pdestrian columns in df_accident
df_accident = pd.merge(df_accident, df_involve_a, on='Accident_no')

len(df_accident)

75229

In [9]:
# Reorganize injury degree in individaul-based data
grouping = [(df_individual.Injury_degree == 1),(df_individual.Injury_degree == 5),
            (df_individual.Injury_degree == 2),(df_individual.Injury_degree == 3)] # 4 == Unknown

# 0:'Death within 24 hours',1:'Death within 30 days',2:'Injury',3:'Non-injury'
new_group = [0,1,2,3]

df_individual['Injury_level'] = np.select(grouping, new_group, default=np.nan)

### Now we shift our focuses to pedestrian-related data
We will first output all individuals in the pedestrian-related accidents.

In [10]:
# All accident_no that involve pedestrian
Pedestrian_accident = df_accident[df_accident.involve_pedestrian > 0]
PA_no = Pedestrian_accident[['Accident_no']]

# All individual that involve in pedestrain related accidents
Pedestrian_accident_individual = pd.merge(df_individual, PA_no, on='Accident_no', how='inner')
len(Pedestrian_accident_individual)

13205

- Now, we will transform some useful columns.
- Some key factors' definition referred to Pai C. et al. in 2019.

In [11]:
## Create a new column to transform the main cause of accident
grouping = [(Pedestrian_accident_individual.Cause_code_major == 15),
            (Pedestrian_accident_individual.Cause_code_major == 23),
            (Pedestrian_accident_individual.Cause_code_major.isin([43,44,67])),
            (Pedestrian_accident_individual.Cause_code_major <= 50),
            (Pedestrian_accident_individual.Cause_code_major <= 60)]

# 1:'Driver - Improper driving at pedestrian crossing', 2:'Driver - Not paying enough attention',
# 3:'Driver - Other causes', 4:'Pedestrians fail to comply with regulations', 5:'Unable to identify'
new_group = [1,2,5,3,4]

Pedestrian_accident_individual['Cause_major_gp'] = np.select(grouping, new_group, default=np.nan)

In [12]:
## Create a new column to transform the weather condition
grouping = [(Pedestrian_accident_individual.Weather.isin([1,6])),
            (Pedestrian_accident_individual.Weather.isin([4,7])),
            (Pedestrian_accident_individual.Weather == 8)]

# 1:'Sunny',2:'Cloudy',3:'Rainy'
new_group = [3,2,1]

Pedestrian_accident_individual['Weather_gp'] = np.select(grouping, new_group)

## Create a new column to transform the Light condition
grouping = [(Pedestrian_accident_individual.Light == 1),(Pedestrian_accident_individual.Light.isin([2,3,4]))]

# 1:'Natural daylight',2:'Night or early morning'
new_group = [1,2]

Pedestrian_accident_individual['Light_gp'] = np.select(grouping, new_group)

In [13]:
## Create a new column to transform the Road surface condition
grouping = [(Pedestrian_accident_individual.Road_condition2 <= 4),
            (Pedestrian_accident_individual.Road_condition2 == 5)]

# 0:'Dry',1:'Slippery'
new_group = [1,0]

Pedestrian_accident_individual['Road_surface'] = np.select(grouping, new_group)

## Create a new column to transform the Road type condition
grouping = [(Pedestrian_accident_individual.Road_type.isin([3,4,5])),
            (Pedestrian_accident_individual.Road_type == 14)]

# 1:'Intersection',2:'Straight road',3:'Other'
new_group = [1, 2]

Pedestrian_accident_individual['Road_type_gp'] = np.select(grouping, new_group, default=3)

## create new columns with Road edges
grouping = [(Pedestrian_accident_individual.Lane_division_3 == 1),
            (Pedestrian_accident_individual.Lane_division_3 == 2)]

# 0:'With Road edge', 1:'Without Road edge'
new_group = [0, 1]

Pedestrian_accident_individual['Road_edge'] = np.select(grouping, new_group)

In [14]:
# merge information with vehicle involve in accident
accident_involve = df_accident[['Accident_no','Fatal','involve_bus','involve_car','involve_scooter',
                                'involve_bicycle','involve_pedestrian','involve_other','involve_people']]
Pedestrian_accident_individual = pd.merge(Pedestrian_accident_individual, accident_involve, on='Accident_no')

In [15]:
## Create a new column about driver's vehicle
Bus_d = ((Pedestrian_accident_individual.involve_bus > 0) 
       & (Pedestrian_accident_individual.involve_car == 0)
       & (Pedestrian_accident_individual.involve_scooter == 0) 
       & (Pedestrian_accident_individual.involve_bicycle == 0)
       & (Pedestrian_accident_individual.involve_other == 0))
Car_d = ((Pedestrian_accident_individual.involve_bus == 0) 
       & (Pedestrian_accident_individual.involve_car > 0)
       & (Pedestrian_accident_individual.involve_scooter == 0) 
       & (Pedestrian_accident_individual.involve_bicycle == 0)
       & (Pedestrian_accident_individual.involve_other == 0))
Sco_d = ((Pedestrian_accident_individual.involve_bus == 0) 
       & (Pedestrian_accident_individual.involve_car == 0)
       & (Pedestrian_accident_individual.involve_scooter > 0) 
       & (Pedestrian_accident_individual.involve_bicycle == 0)
       & (Pedestrian_accident_individual.involve_other == 0))
Bic_d = ((Pedestrian_accident_individual.involve_bus == 0) 
       & (Pedestrian_accident_individual.involve_car == 0)
       & (Pedestrian_accident_individual.involve_scooter == 0) 
       & (Pedestrian_accident_individual.involve_bicycle > 0)
       & (Pedestrian_accident_individual.involve_other == 0))
Nan_d = ((Pedestrian_accident_individual.involve_bus == 0) 
       & (Pedestrian_accident_individual.involve_car == 0)
       & (Pedestrian_accident_individual.involve_scooter == 0) 
       & (Pedestrian_accident_individual.involve_bicycle == 0)
       & (Pedestrian_accident_individual.involve_other == 0))

Conditions = [Bus_d, Car_d, Sco_d, Bic_d, Nan_d]

# 1:'Bus or Truck', 2:'Sedan or Small Truck', 3:'Motorcycle or Scooter', 4:'Bicycle', 5:'Other or mix'
values = [1,2,3,4,np.nan]

Pedestrian_accident_individual['Driver_vehicle'] = np.select(Conditions, values, default=5)

In [16]:
## create new columns with season
grouping = [(Pedestrian_accident_individual.Month.isin([3,4,5])),
            (Pedestrian_accident_individual.Month.isin([6,7,8])),
            (Pedestrian_accident_individual.Month.isin([9,10,11])),
            (Pedestrian_accident_individual.Month.isin([12,1,2]))]

# 1:'Spring', 2:'Summer', 3:'Autumn', 4:'Winter'
new_group = [1,2,3,4]

Pedestrian_accident_individual['Season'] = np.select(grouping, new_group)

## create new columns with day of the week: weekday or weekend
Pedestrian_accident_individual['Date'] = pd.to_datetime(Pedestrian_accident_individual[['Year', 'Month', 'Day']])
Pedestrian_accident_individual['Weekday'] = Pedestrian_accident_individual['Date'].dt.weekday
Pedestrian_accident_individual['is_Weekend'] = (Pedestrian_accident_individual['Weekday'] >= 5).astype(int)

## create new columns with hours in the day
grouping = [(Pedestrian_accident_individual.Hours < 7),
            ((Pedestrian_accident_individual.Hours >= 7) & (Pedestrian_accident_individual.Hours < 9)),
            ((Pedestrian_accident_individual.Hours >= 9) & (Pedestrian_accident_individual.Hours < 17)),
            ((Pedestrian_accident_individual.Hours >= 17) & (Pedestrian_accident_individual.Hours < 19)),
            (Pedestrian_accident_individual.Hours >= 19)]

# 1:'Midnight (12am-7am)', 2:'Morning Rush hour (7am-9am)', 3:'Midday (9am-5pm)',
# 4:'Afternoon Rush hour (5pm-7pm)', 5:'Evening and Night(7pm-12am)'
new_group = [1,2,3,4,5]

Pedestrian_accident_individual['Hour_gp'] = np.select(grouping, new_group)

In [17]:
## create new columns with it's location
grouping = [(Pedestrian_accident_individual.District.isin([8,9])),
            (Pedestrian_accident_individual.District.isin([1,2,3,5])),
            (Pedestrian_accident_individual.District.isin([10])),
            (Pedestrian_accident_individual.District.isin([11,12])),
            (Pedestrian_accident_individual.District.isin([4,6,7]))]

# 1:'North', 2:'West', 3:'South', 4:'East', 5:'Central'
new_group = [1,2,3,4,5]

Pedestrian_accident_individual['Location'] = np.select(grouping, new_group)

In [18]:
# Create Pedestrian-related accident data
PA_col = ['Year', 'Month', 'Day', 'Hours', 'Minutes', 'Category',
          'Accident_no', 'Cause_major_gp', 'Weather_gp', 'Light_gp', 
          'Road_surface', 'Road_type_gp', 'Season', 'Fatal',
          'involve_bus', 'involve_car', 'involve_scooter', 'involve_bicycle',
          'involve_pedestrian', 'involve_other', 'involve_people',
          'Driver_vehicle', 'is_Weekend', 'Hour_gp', 'Location', 'Road_edge']

PA_pre = Pedestrian_accident_individual[PA_col]
Pedestrian_accident = PA_pre.drop_duplicates(subset='Accident_no')
len(Pedestrian_accident)

6251

### Now, we will create some driver's information for Pedestrian individual data.

In [19]:
# Output rows that are not people
condition = ((Pedestrian_accident_individual['Parties_category'].isin(['H01','H02','H03']))
            | (Pedestrian_accident_individual['Parties_category'].isna()))

driver_col = ['Parties_category', 'Age', 'Sex', 'Driving_qualification', 'Drinking',
              'hit_and_run', 'Category_grouping', 'Accident_no',
              'involve_bus', 'involve_car', 'involve_scooter', 'involve_bicycle',
              'involve_pedestrian', 'involve_other', 'involve_people', 'Driver_vehicle']

Pedestrian_accident_individual['Sex'] = Pedestrian_accident_individual['Sex'].astype('object')
Pedestrian_driver = Pedestrian_accident_individual[~condition][driver_col]

### Transform driver's variables
## Driving without license
Drive_license = [(Pedestrian_driver.Driving_qualification.isin([1,8])),
                 (Pedestrian_driver.Driving_qualification.isin([2,3,4,5,6]))]
# 0:'With proper license', 1:'Without proper license'
Drive_license_group = [0, 1]

Pedestrian_driver['nDrive_license'] = np.select(Drive_license, Drive_license_group, default=np.nan)

## Drinking
Drinking = [(Pedestrian_driver.Drinking.isin([1,2,10])),
            (Pedestrian_driver.Drinking.isin([3,4,5,6,7,8]))]
# 0:'No Alcohol use', 1:'Alcohol use'
Drinking_group = [0, 1]

Pedestrian_driver['Driver_drinking'] = np.select(Drinking, Drinking_group, default=np.nan)

## Count how many vehicle involve in the accident
Pedestrian_driver['Driver_num'] = (Pedestrian_driver.involve_bus + Pedestrian_driver.involve_car +
                                   Pedestrian_driver.involve_scooter + Pedestrian_driver.involve_bicycle +
                                   Pedestrian_driver.involve_other)

Pedestrian_driver['Multiple_driver'] = (Pedestrian_driver['Driver_num'] > 1)

## driver's age output, if there's multiple drivers than take mean of the age
Driver_age = Pedestrian_driver.groupby('Accident_no')['Age'].mean().reset_index(name='Driver_age')

## driver's information output
Driver_info = Pedestrian_driver.groupby('Accident_no')[['nDrive_license','Driver_drinking',
                                                        'hit_and_run']].max().reset_index()
Driver_info = Driver_info.astype('object')

### Merge Driver's info
Driver_info_col = ['Accident_no', 'Driver_vehicle', 'Driver_num', 'Multiple_driver']
Pedestrian_driver_col = Pedestrian_driver[Driver_info_col].drop_duplicates(subset=['Accident_no'])
Pedestrian_driver_info1 = pd.merge(Pedestrian_driver_col, Driver_age, on='Accident_no', how='left')
Pedestrian_driver_info2 = pd.merge(Pedestrian_driver_info1, Driver_info, on='Accident_no', how='left')

In [20]:
## Driver's sex need more work to output
# Accident that only involve one driver or multiple driver
Pedestrian_driver1 = Pedestrian_driver[Pedestrian_driver.Multiple_driver == False][['Accident_no','Sex']]
Pedestrian_driver2 = Pedestrian_driver[Pedestrian_driver.Multiple_driver == True][['Accident_no','Sex']]

# If only one driver
Pedestrian_driver1['Driver_sex'] = Pedestrian_driver1['Sex']

# With multiple driver accident
Pedestrian_driver2 = Pedestrian_driver2.dropna()
Pedestrian_driver2 = Pedestrian_driver2.drop_duplicates(subset=['Accident_no','Sex'])

Pedestrian_driver2_count = Pedestrian_driver2.groupby('Accident_no').size().reset_index(name='Count')
Pedestrian_driver2_count1 = Pedestrian_driver2_count[Pedestrian_driver2_count.Count == 1]
Pedestrian_driver2_count2 = Pedestrian_driver2_count[Pedestrian_driver2_count.Count > 1]

# With only one sex involve
Pedestrian_driver_M1 = pd.merge(Pedestrian_driver2, Pedestrian_driver2_count1, on='Accident_no', how='right')
Pedestrian_driver_M1['Driver_sex'] = Pedestrian_driver_M1['Sex']

# With both sex involve
Pedestrian_driver_M2 = Pedestrian_driver2_count2.copy()
Pedestrian_driver_M2['Driver_sex'] = '3'

# Driver's gender information
Driver_gender = pd.concat([Pedestrian_driver1,Pedestrian_driver_M1,Pedestrian_driver_M2])[['Accident_no',
                                                                                           'Driver_sex']]

## Merge with driver's information
Pedestrian_driver_info = pd.merge(Pedestrian_driver_info2, Driver_gender, on='Accident_no')

In [21]:
# Create Pedestrian individual data merge with driver's information
condition = ((Pedestrian_accident_individual['Parties_category']=='H01')
           & (Pedestrian_accident_individual['Injury_level'].notna()))
PI_col = ['Age', 'Sex', 'Accident_no', 'Injury_level', 'Cause_major_gp', 
          'Weather_gp', 'Light_gp', 'Road_surface', 'Road_type_gp', 'Season', 
          'involve_bus', 'involve_car', 'involve_scooter', 'involve_bicycle', 'involve_other', 
          'involve_people', 'is_Weekend', 'Hour_gp', 'Location','Road_edge']

Pedestrian_individual = Pedestrian_accident_individual[condition][PI_col]
Pedestrian_individual = pd.merge(Pedestrian_individual, Pedestrian_driver_info, on='Accident_no', how='inner')

len(Pedestrian_individual)

6329

In [22]:
# Saving datasets as Pickle for later analysis

df_accident.to_pickle('df_accident.pkl')
df_individual.to_pickle('df_individual.pkl')
Pedestrian_accident.to_pickle('Pedestrian_accident.pkl')
Pedestrian_individual.to_pickle('Pedestrian_individual.pkl')

*Now we prepare all the datasets and ready for the later analysis.*
- df_accident : Contain all accident-based data with vehicles involve.
- df_individaul : Contain all individaul-based data with its vehicle use.
- Pedestrian_accident : Contian all pedestrian-related accident with key factors.
- Pedestrian_individual : Contain all pedestrian data with its accident condition and driver's information.